# SVIPro Tutorial: Introduction to Spatial Sampling

**SVI Research Protocol & Optimization**

This notebook introduces the basics of SVIPro - A standardized framework for reproducible Street View Imagery sampling design.

---

## Table of Contents

1. [Installation and Setup](#1-installation-and-setup)
2. [Basic Concepts](#2-basic-concepts)
3. [Grid Sampling](#3-grid-sampling)
4. [Road Network Sampling](#4-road-network-sampling)
5. [Quality Metrics](#5-quality-metrics)
6. [Data Export](#6-data-export)
7. [Visualization](#7-visualization)
8. [Best Practices](#8-best-practices)

---

## 1. Installation and Setup

First, let's import the necessary libraries:

In [ ]:
# Core SVIPro imports
import svipro
from svipro import (
    GridSampling, 
    RoadNetworkSampling, 
    SamplingConfig
)

# Geospatial libraries
import geopandas as gpd
from shapely.geometry import box, Polygon

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Data handling
import pandas as pd
import numpy as np

# Check version
print(f"SVIPro version: {svipro.__version__}")
print(f"Authors: {svipro.__author__}")

---

## 2. Basic Concepts

### What is SVIPro?

SVIPro provides **scientific, reproducible, and documented** spatial sampling methods for urban studies using Street View Imagery.

### Key Features

- ✅ **Multiple Sampling Strategies**: Grid, road network, and more
- ✅ **Reproducibility**: Same parameters = same results (using seeds)
- ✅ **Quality Assessment**: Built-in metrics and validation
- ✅ **Standardized Output**: GeoJSON with complete metadata

### Core Components

1. **SamplingConfig**: Configuration parameters (spacing, CRS, seed)
2. **SamplingStrategy**: Algorithm for generating sample points
3. **GeoDataFrame**: Output format with spatial data

---

## 3. Grid Sampling

Grid sampling is the simplest and most transparent method. It creates a regular grid of sample points.

### 3.1 Define Your Study Area

First, create a boundary for your area of interest (AOI):

In [ ]:
# Create a rectangular boundary (in degrees, roughly 10km x 10km)
# This example uses Milan, Italy coordinates
milan_boundary = box(9.15, 45.42, 9.25, 45.52)

print(f"Boundary type: {type(milan_boundary)}")
print(f"Boundary area: {milan_boundary.area:.4f} square degrees")
print(f"Bounds: {milan_boundary.bounds}")

### 3.2 Configure Sampling Parameters

Create a `SamplingConfig` object with your parameters:

In [ ]:
# Create configuration
config = SamplingConfig(
    spacing=100.0,      # 100 meters between points
    crs="EPSG:4326",   # WGS84 coordinate system
    seed=42             # For reproducibility
)

print(f"Spacing: {config.spacing} meters")
print(f"CRS: {config.crs}")
print(f"Seed: {config.seed}")

### 3.3 Generate Grid Sample Points

Now create the sampling strategy and generate points:

In [ ]:
# Create grid sampling strategy
grid_strategy = GridSampling(config)

# Generate sample points
grid_points = grid_strategy.generate(milan_boundary)

# Display results
print(f"Generated {len(grid_points)} sample points")
print(f"\nColumns in output:")
print(grid_points.columns.tolist())
print(f"\nFirst 5 points:")
print(grid_points.head())

### 3.4 Visualize Grid Sample Points

In [ ]:
# Plot the grid sampling results
fig, ax = plt.subplots(figsize=(10, 10))

# Plot boundary
gpd.GeoSeries([milan_boundary]).plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2)

# Plot sample points
grid_points.plot(ax=ax, markersize=10, color='blue', alpha=0.6)

ax.set_title('Grid Sampling Results (100m spacing)', fontsize=14)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.tight_layout()
plt.show()

---

## 4. Road Network Sampling

Road network sampling places points along actual roads, which is more realistic for street view studies.

### 4.1 Configure Road Network Sampling

**Note**: This requires an internet connection to download road data from OpenStreetMap.

In [ ]:
# Create configuration for road network sampling
road_config = SamplingConfig(
    spacing=100.0,      # 100 meters between points
    crs="EPSG:4326",
    seed=42
)

# Create road network sampling strategy
road_strategy = RoadNetworkSampling(
    road_config,
    network_type='drive',    # Use drivable roads only
    # road_types={'primary', 'secondary'}  # Optional: filter by road type
)

print("Road network strategy created")
print(f"Network type: {road_strategy.network_type}")

### 4.2 Generate Road Network Sample Points

This may take a moment as it downloads road data from OpenStreetMap:

In [ ]:
# Generate road network sample points
print("Downloading road network from OpenStreetMap...")
road_points = road_strategy.generate(milan_boundary)

print(f"\nGenerated {len(road_points)} sample points along roads")
print(f"\nFirst 5 points:")
print(road_points.head())

### 4.3 Calculate Road Network Metrics

In [ ]:
# Get road network specific metrics
road_metrics = road_strategy.calculate_road_network_metrics()

print("Road Network Metrics:")
print(f"  Sample points: {road_metrics['n_points']}")
print(f"  Network edges: {road_metrics['n_edges']}")
print(f"  Network nodes: {road_metrics['n_nodes']}")
print(f"  Total road length: {road_metrics['total_road_length_km']:.2f} km")
print(f"  Average node degree: {road_metrics['avg_degree']:.2f}")

if road_metrics['road_type_distribution']:
    print(f"\nRoad type distribution:")
    for road_type, count in sorted(road_metrics['road_type_distribution'].items()):
        print(f"  {road_type}: {count} points")

### 4.4 Visualize Road Network Sample Points

In [ ]:
# Plot road network sampling results
fig, ax = plt.subplots(figsize=(10, 10))

# Plot boundary
gpd.GeoSeries([milan_boundary]).plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2)

# Plot sample points, color by highway type
if 'highway' in road_points.columns:
    road_points.plot(ax=ax, column='highway', markersize=15, alpha=0.7, legend=True, categorical=True)
else:
    road_points.plot(ax=ax, markersize=15, color='green', alpha=0.7)

ax.set_title('Road Network Sampling Results (100m spacing)', fontsize=14)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.tight_layout()
plt.show()

---

## 5. Quality Metrics

SVIPro provides built-in metrics to assess sampling quality.

In [ ]:
# Calculate coverage metrics for grid sampling
grid_metrics = grid_strategy.calculate_coverage_metrics()

print("Grid Sampling Metrics:")
print(f"  Number of points: {grid_metrics['n_points']}")
print(f"  Coverage area: {grid_metrics['area_km2']:.4f} km²")
print(f"  Sampling density: {grid_metrics['density_pts_per_km2']:.2f} pts/km²")
print(f"  Bounds: {grid_metrics['bounds']}")
print(f"  CRS: {grid_metrics['crs']}")

### 5.1 Compare Strategies

In [ ]:
# Compare metrics between strategies
comparison = pd.DataFrame({
    'Grid Sampling': [
        grid_metrics['n_points'],
        grid_metrics['density_pts_per_km2'],
        'Uniform'
    ],
    'Road Network Sampling': [
        road_metrics['n_points'],
        road_metrics['n_points'] / grid_metrics['area_km2'],
        'Road-following'
    ]
}, index=['Points', 'Density (pts/km²)', 'Pattern'])

print("Strategy Comparison:")
print(comparison)

---

## 6. Data Export

Export your sampling results with complete metadata.

In [ ]:
# Export grid sampling to GeoJSON with metadata
grid_strategy.to_geojson("grid_samples_milan.geojson", include_metadata=True)
print("✓ Grid samples exported to: grid_samples_milan.geojson")

# Export road network sampling to GeoJSON
road_strategy.to_geojson("road_samples_milan.geojson", include_metadata=True)
print("✓ Road network samples exported to: road_samples_milan.geojson")

### 6.1 Export Config as Dictionary

In [ ]:
# Export configuration as dictionary (for JSON/YAML)
config_dict = config.to_dict()

print("Configuration as dictionary:")
for key, value in config_dict.items():
    if key != 'boundary':  # Skip boundary WKT (too long)
        print(f"  {key}: {value}")

---

## 7. Visualization

SVIPro provides built-in visualization tools.

In [ ]:
# Import visualization functions
from svipro import plot_coverage_statistics, plot_spatial_distribution

# Create coverage statistics plot
fig = plot_coverage_statistics(
    grid_points,
    title='Grid Sampling Coverage Statistics'
)
plt.show()

### 7.1 Spatial Distribution Plot

In [ ]:
# Create spatial distribution plot
fig, ax = plt.subplots(figsize=(10, 8))
plot_spatial_distribution(grid_points, milan_boundary, ax=ax)
ax.set_title('Spatial Distribution of Grid Sample Points')
plt.show()

---

## 8. Best Practices

### 8.1 Choosing the Right Strategy

| Strategy | Best For | Pros | Cons |
|----------|----------|------|------|
| **Grid** | General studies, baseline comparison | Simple, reproducible, uniform coverage | May not follow accessible routes |
| **Road Network** | Street view, transportation studies | Realistic placement, follows roads | Requires OSM data, less uniform |

### 8.2 Setting Parameters

**Spacing Guidelines:**
- **Urban areas**: 50-100m (dense coverage)
- **Suburban areas**: 100-200m (moderate coverage)
- **Rural areas**: 200-500m (sparse coverage)

**Reproducibility:**
- Always set a `seed` for reproducible results
- Document your configuration parameters
- Use the same CRS for comparison studies

### 8.3 Common Issues and Solutions

**Issue 1: No points generated**

```python
# Solution: Check boundary size vs spacing
# Try: (1) Larger boundary, (2) Smaller spacing
```

**Issue 2: OSM download fails**

```python
# Solution: Check internet connection and boundary validity
# Try: (1) Different network_type ('walk' vs 'drive'),
#      (2) Smaller boundary area
```

**Issue 3: Points outside boundary**

```python
# Solution: Check CRS compatibility
# Use projected CRS (e.g., EPSG:3857) for accurate spacing
```

---

## Summary

In this tutorial, you learned:

1. ✅ How to install and import SVIPro
2. ✅ How to create sampling configurations
3. ✅ How to generate grid and road network samples
4. ✅ How to calculate quality metrics
5. ✅ How to export results with metadata
6. ✅ How to visualize sampling results

### Next Steps

- Try different spacing values
- Compare multiple strategies on the same area
- Use your own study area boundary
- Integrate with your research workflow

### Resources

- **Documentation**: `docs/` folder
- **API Reference**: `docs/api_reference.md`
- **Case Studies**: `docs/case_studies/`
- **GitHub**: https://github.com/GuojialeGeographer/GProcessing2025

---

**Authors**: Jiale Guo & Mingfeng Tang  
**Institution**: Politecnico di Milano  
**Course**: Geospatial Processing  
**Date**: 2025-01-23